In [2]:
from dotenv import load_dotenv

from pathlib import Path
import os 
import sys
import numpy as np
import json

load_dotenv()
GEMINI_KEY = os.getenv('GEMINI_KEY')

root = Path().resolve().parent

root_str = str(root)
if root_str not in sys.path:
    sys.path.insert(0, root_str)

if Path.cwd() != root:
    os.chdir(root)



from google import genai

genai_client = genai.Client(
    api_key=GEMINI_KEY,
)

In [16]:

response = genai_client.models.generate_content(
    model="gemini-2.5-flash-preview-05-20",
    contents="Roses are red...",
)

response.candidates[0].content.parts[0].text

'Violets are blue,\nSugar is sweet,\nAnd so are you.'

### Trial

In [2]:
import json

string = "```json\n{\"api_name\":\"RegisterUser\",\"parameters\":{\"username\":\"johnsmith123\",\"password\":\"1234password\",\"email\":\"johnsmith123@example.com\"}}\n```"

json.loads(string)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
with open("data/generated/1-rotowire/generated.json", "r") as f:
    data = json.load(f)

json.loads(data[0]["generated_output"].replace("```json", "").replace("```", ""))

{'teams': [{'team': 'Atlanta Hawks',
   'total_points': 107,
   'points_in_2nd_quarter': 31,
   'percentage_of_field_goals': 49,
   'number_of_team_assists': 29,
   'wins': 9,
   'losses': 2},
  {'team': 'Milwaukee Bucks',
   'total_points': 100,
   'points_in_2nd_quarter': 9,
   'wins': 5,
   'losses': 5}],
 'players': [{'player': 'Giannis Antetokounmpo',
   'points': 26,
   'total_rebounds': 15,
   'assists': 7},
  {'player': 'Jabari Parker',
   'points': 23,
   'total_rebounds': 8,
   'assists': 3,
   'steals': 3},
  {'player': 'Tony Snell', 'points': 11, 'total_rebounds': 4, 'assists': 3},
  {'player': 'Mirza Teletovic',
   'points': 14,
   'field_goals_made': 5,
   'field_goals_attempted': 10},
  {'player': 'Paul Millsap', 'points': 21, 'total_rebounds': 8, 'assists': 3},
  {'player': 'Mike Muscala', 'points': 16, 'total_rebounds': 5, 'assists': 3},
  {'player': 'Kyle Korver', 'points': 11, 'total_rebounds': 6, 'assists': 5}]}